In [2]:
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
plt.rcParams['font.family'] = 'NanumGothicCoding'
plt.rcParams['axes.unicode_minus'] = False


In [3]:
df = pd.read_csv('../../data/team/Pollution 전처리/해양환경공단_해양폐기물정화사업 현황_20181231.csv', encoding = 'ANSI', engine = 'python')
df.to_csv('../../data/team/폐기물정화사업.csv',encoding = 'utf-8',index=False)

In [4]:
# 기존 데이터에 정화사업의 면적, 수거물량을 병합
df = df.rename(columns={'사업명(위치)':'위치','수거물량 실적(톤)':'수거','정화면적 실적(ha)':'정화면적'})
df['정화면적'] = df['정화면적'].astype(float)
df['시군'] = '' 
df['시군'] = df['위치'].str.split(' ').str[0]

In [5]:
df_all = pd.read_csv('../../data/team/수질+수거량.csv')


In [6]:
# 구군의 데이터 중 '섬 진 강'의 형식을 가지는 데이터 전처리 
li_a = list(df_all['구군'])
li_b = []
for i in li_a:
    a = i.replace(' ','')
    li_b.append(a)
df_all['구군'] = li_b

In [ ]:
gu = df_all['구군'].unique()

In [47]:
# 기존파일의 생태구역을 먼저 넣은 뒤 부족한 지역을 구군으로 지정해서 넣음
# 기준은 해양환경관리법의 기준에 따름
for i in range(len(df)):
    for j in gu:
        if df.loc[i,'위치'].find(j) != -1:
            df.loc[i,'생태구역'] = df_all[df_all['구군'] == j]['생태구역'].iloc[0]

for i in df[df['생태구역'].isnull()].index:
    al = df.loc[i,'위치']
    if (al.find('삼천포') != -1)|(al.find('거제') != -1)|(al.find('하동') != -1)|(al.find('경남') != -1):
        df.loc[i,'생태구역'] = '대한해협'
        
    elif (al.find('서천') != -1)|(al.find('당진') != -1)|(al.find('충남') != -1)|(al.find('영종도') != -1)|(al.find('평택') != -1):
        df.loc[i,'생태구역'] = '서해중부'
        
    elif (al.find('당진') != -1)|(al.find('장흥') != -1)|(al.find('전북') != -1)|(al.find('영암') != -1)|(al.find('전남') != -1)|(al.find('흑산도') != -1):
        df.loc[i,'생태구역'] = '서남해역'
        
    elif (al.find('경북') != -1)|(al.find('독도') != -1)|(al.find('포항') != -1):
        df.loc[i,'생태구역'] = '동해'

In [49]:
# 생태구역별 평균
# 시군은 데이터가 적어 년도별 분류가 불가능
df1 = df.groupby('시군').mean()
df2 = df.groupby(['년도','생태구역']).mean()

df1_2 = pd.DataFrame(columns=['년','생태구역','수거량','면적'])
idx = 0
for i in df2.index:
    df1_2.loc[idx,'년'] = i[0]
    df1_2.loc[idx,'생태구역'] = i[1]
    df1_2.loc[idx,'수거량'] = df2.loc[i[0],i[1]][0]
    df1_2.loc[idx,'면적'] = df2.loc[i[0],i[1]][1]
    idx +=1

In [50]:
# 다른 해협은 15~18년도 자료가 있으나 제주도는 16,17년도 자료가 없음
# 이를 해결하기 위해서 평균값으로 널값 채움
avg = df1_2[df1_2['생태구역'] == '제주'].mean()
df1_2['년'] = df1_2['년'].astype(float)

df1_2.loc[18] = [2016,'제주',avg[1], avg[2]]
df1_2.loc[19] = [2017,'제주',avg[1], avg[2]]

In [ ]:
df_all['수거량2']=''
df_all['정화면적']=''

# 생태구역을 기준
for i in df_all.index:
    
    year = round(df_all.loc[i,'년월']/100)
    area = df_all.loc[i,'생태구역'] 
    
    for j in df1_2['년'].unique():
        for z in df1_2['생태구역'].unique():
            if (year==j)&(area==z):
                tem = df1_2[(df1_2['년'] ==j)&(df1_2['생태구역']==z)]
                df_all.loc[i,'수거량2'] = tem['수거량'].iloc[0]
                df_all.loc[i,'정화면적'] = tem['면적'].iloc[0]
      

In [68]:
       
# 구군을 기준    
for i in df_all.index:
    year = int(df_all.loc[i,'년월']/100)
    area = df_all.loc[i,'구군'] 
    for j in df.index:
        # 기존데이터의 구군이 정화사업데이터의 위치에 있을 경우
        if (year == df.loc[j,'년도'])&(df.loc[j,'위치'].find(area) != -1):
            df_all.loc[i,'수거량2'] = df.loc[j,'수거']
            df_all.loc[i,'정화면적'] = df.loc[j,'정화면적']   
            break
            

In [75]:
df_all.to_csv('../../data/team/수질+수거량+정화.csv', encoding='utf-8', index=False)
df_all.to_csv('../../data/team/수질+수거량+정화_R.csv', encoding='ANSI', index=False)